<a href="https://colab.research.google.com/github/kartik5465/deeplearning/blob/master/(4_to_1_RNN)_to_show_that_the_quantity_of_rain_on_a_certain_day_also_depends_on_the_values_of_the_previous_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
X = []
X.append([1,0,0,0])
X.append([0,1,0,0])
X.append([0,0,1,0])
X.append([0,0,0,1])
X.append([0,0,0,1])
X.append([1,0,0,0])
X.append([0,1,0,0])
X.append([0,0,1,0])
X.append([0,0,0,1])

y = [0.20, 0.30, 0.40, 0.50, 0.05, 0.10, 0.20, 0.30, 0.40]

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_der(x):
    return 1.0 - x**2

In [ ]:
layers = []
# 4 input variables, 10 hidden units and 1 output variable
n_units = (4, 16, 1)
n_layers = len(n_units)

layers.append(np.ones(n_units[0]+1+n_units[1]))
for i in range(1, n_layers):
    layers.append(np.ones(n_units[i]))

weights = []
for i in range(n_layers-1):
    weights.append(np.zeros((layers[i].size, layers[i+1].size)))

weights_delta = [0,]*len(weights)

In [ ]:
def forwards(data):
    layers[0][:n_units[0]] = data
    layers[0][n_units[0]:-1] = layers[1]

    # Propagate the data forwards
    for i in range(1, n_layers):
        layers[i][...] = sigmoid(np.dot(layers[i-1], weights[i-1]))

    return layers[-1]

In [ ]:
def backwards(target, learning_rate=0.1, momentum=0.1):
    deltas = []
    error = target - layers[-1]
    delta = error * sigmoid_der(layers[-1])
    deltas.append(delta)

    # Determine error in hidden layers
    for i in range(n_layers-2, 0, -1):
        delta = np.dot(deltas[0], weights[i].T) * sigmoid_der(layers[i])
        deltas.insert(0, delta)

    # Update weights
    for i in range(len(weights)):
        layer = np.atleast_2d(layers[i])
        delta = np.atleast_2d(deltas[i])
        weights_delta_temp = np.dot(layer.T, delta)
        weights[i] += learning_rate*weights_delta_temp + momentum*weights_delta[i]
        weights_delta[i] = weights_delta_temp

    return (error**2).sum()

In [ ]:
n_epochs = 10000

for i in range(n_epochs):
    loss = 0
    for j in range(len(X)):
        forwards(X[j])
        backwards(y[j])
        loss += (y[j]-forwards(X[j]))**2
    if i%1000 == 0: print('epoch {} - loss: {:04.4f}'.format(i, loss[0]))

epoch 0 - loss: 0.3116
epoch 1000 - loss: 0.1660
epoch 2000 - loss: 0.1801
epoch 3000 - loss: 0.1877
epoch 4000 - loss: 0.1914
epoch 5000 - loss: 0.1922
epoch 6000 - loss: 0.1920
epoch 7000 - loss: 0.1916
epoch 8000 - loss: 0.1913
epoch 9000 - loss: 0.1912


In [ ]:
for i in range(len(X)):
    pred = forwards(X[i])
    loss = (y[i]-pred)**2
    print('X: {}; y: {:04.2f}; pred: {:04.2f}'.format(X[i], y[i], pred[0]))

X: [1, 0, 0, 0]; y: 0.20; pred: 0.15
X: [0, 1, 0, 0]; y: 0.30; pred: 0.39
X: [0, 0, 1, 0]; y: 0.40; pred: 0.29
X: [0, 0, 0, 1]; y: 0.50; pred: 0.34
X: [0, 0, 0, 1]; y: 0.05; pred: 0.30
X: [1, 0, 0, 0]; y: 0.10; pred: 0.16
X: [0, 1, 0, 0]; y: 0.20; pred: 0.38
X: [0, 0, 1, 0]; y: 0.30; pred: 0.30
X: [0, 0, 0, 1]; y: 0.40; pred: 0.33
